In [1]:
import json, requests
!pip install urllib2
import  urllib.request as urllib2
from bs4 import BeautifulSoup
import csv

  Could not find a version that satisfies the requirement urllib2 (from versions: )
No matching distribution found for urllib2


In [2]:
#Open URL and read it
url = urllib2.urlopen('http://www.boxofficemojo.com/alltime/domestic.htm')
content = url.read()

In [3]:
soup = BeautifulSoup(content)

C:\Users\Michael\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Michael\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [4]:
movies_data = soup.findAll('font',{'size':2})

In [5]:
movies_data

[<font face="Verdana" size="2"><b><font face="Verdana" size="4">#1100</font> - <a href="/alltime/domestic.htm?page=2&amp;p=.htm">#101200</a> - <a href="/alltime/domestic.htm?page=3&amp;p=.htm">#201300</a> - <a href="/alltime/domestic.htm?page=4&amp;p=.htm">#301400</a> - <a href="/alltime/domestic.htm?page=5&amp;p=.htm">#401500</a> ...</b></font>,
 <font size="2"><a href="/alltime/domestic.htm?page=1&amp;sort=gross&amp;order=ASC&amp;p=.htm"><b>Rank</b></a></font>,
 <font size="2"><a href="/alltime/domestic.htm?page=1&amp;sort=title&amp;order=ASC&amp;p=.htm">Title(click to view)</a></font>,
 <font size="2"><a href="/alltime/domestic.htm?page=1&amp;sort=studio&amp;order=ASC&amp;p=.htm">Studio</a></font>,
 <font size="2"><a href="/alltime/domestic.htm?page=1&amp;sort=gross&amp;order=ASC&amp;p=.htm"><b>Lifetime Gross</b></a></font>,
 <font size="2"><a href="/alltime/domestic.htm?page=1&amp;sort=year&amp;order=DESC&amp;p=.htm">Year^</a></font>,
 <font size="2">1</font>,
 <font size="2">

In [6]:
movie_dict = {}
for i in range (100):
    rank = movies_data[6 + 5*i].get_text()
    name = movies_data[7+ 5*i].find('b').get_text()
    studio = movies_data[8+ 5*i].find('a').get_text()
    box_office = movies_data[9+ 5*i].find('b').get_text()
    year = movies_data[10 + 5*i].get_text().rstrip('^')
    movie_dict[rank] = {} 
    
    #handle exceptions 
    if name == "Marvel's The Avengers":
        name = 'The Avengers'
    elif name == 'Beauty and the Beast (2017)':
        name  = 'Beauty and the Beast'
    elif name == 'The Jungle Book (2016)':
        name = 'The Jungle Book'
    elif name == 'Alice in Wonderland (2010)':
        name = 'Alice in Wonderland'
    movie_dict[rank]['Name'] = name


movie_dict

{'1': {'Name': 'Star Wars: The Force Awakens'},
 '10': {'Name': 'Star Wars: Episode I - The Phantom Menace'},
 '100': {'Name': 'Captain America: The Winter Soldier'},
 '11': {'Name': 'Star Wars'},
 '12': {'Name': 'Avengers: Age of Ultron'},
 '13': {'Name': 'The Dark Knight Rises'},
 '14': {'Name': 'Shrek 2'},
 '15': {'Name': 'E.T.: The Extra-Terrestrial'},
 '16': {'Name': 'The Hunger Games: Catching Fire'},
 '17': {'Name': "Pirates of the Caribbean: Dead Man's Chest"},
 '18': {'Name': 'The Lion King'},
 '19': {'Name': 'Toy Story 3'},
 '2': {'Name': 'Avatar'},
 '20': {'Name': 'Wonder Woman'},
 '21': {'Name': 'Iron Man 3'},
 '22': {'Name': 'Captain America: Civil War'},
 '23': {'Name': 'The Hunger Games'},
 '24': {'Name': 'Spider-Man'},
 '25': {'Name': 'Jurassic Park'},
 '26': {'Name': 'Transformers: Revenge of the Fallen'},
 '27': {'Name': 'Frozen'},
 '28': {'Name': 'Guardians of the Galaxy Vol. 2'},
 '29': {'Name': 'Harry Potter and the Deathly Hallows Part 2'},
 '3': {'Name': 'Titanic

In [17]:
#Write CSV file
with open("my_movie.csv", "w",newline = '') as toWrite:
    writer = csv.writer(toWrite, delimiter=",")
    writer.writerow([ "Rank","Name", "Box Office", "Studio","Year"])
    rank=movie_dict.keys()
    rank= map(int, rank)
    #rank.sort()
    for x in rank:
        x=str(x)
        writer.writerow([x, movie_dict[x]['Name'],
                                 movie_dict[x]['Box Office'], 
                                 movie_dict[x]["Studio"], 
                                 movie_dict[x]["Year"],])

In [18]:
#Import CSV file to open
import pandas
movie_df = pandas.read_csv('my_movie.csv')
movie_df.head()

,Rank,Name,Box Office,Studio,Year
0,1,Star Wars: The Force Awakens,"$936,662,225",BV,2015
1,2,Avatar,"$760,507,625",Fox,2009
2,3,Titanic,"$658,672,302",Par.,1997
3,4,Jurassic World,"$652,270,625",Uni.,2015
4,5,The Avengers,"$623,357,910",BV,2012


In [19]:
# movies_list = []
imdb_movies ={}
baseurl = "http://www.omdbapi.com/?t=" #only submitting the title parameter
# example url http://www.omdbapi.com/?t=Titanic&apikey=f35fdf4
for i in range(movie_df.shape[0]):
    movieTitle = str(movie_df.iloc[i,1])
#     movie_list.append(movieTitle)
    response = requests.get(baseurl + movieTitle + '&apikey=f35fdf4&y=' + str(movie_df.iloc[i,4]))
    if response.status_code == 200:
        imdb_movies[movieTitle] = json.loads(response.text)
    else:
        raise ValueError("Bad request!")
        print(i)

In [20]:
# check if there is an inconsistence
for key,value in imdb_movies.items():
    try:
        if imdb_movies[key]['Error'] == 'Movie not found!':
            print(key)
    except KeyError:
        pass

In [21]:
imdb_movies['Avatar']

{'Actors': 'Sam Worthington, Zoe Saldana, Sigourney Weaver, Stephen Lang',
 'Awards': 'Won 3 Oscars. Another 85 wins & 128 nominations.',
 'BoxOffice': '$749,700,000',
 'Country': 'UK, USA',
 'DVD': '22 Apr 2010',
 'Director': 'James Cameron',
 'Genre': 'Action, Adventure, Fantasy',
 'Language': 'English, Spanish',
 'Metascore': '83',
 'Plot': 'A paraplegic marine dispatched to the moon Pandora on a unique mission becomes torn between following his orders and protecting the world he feels is his home.',
 'Poster': 'https://images-na.ssl-images-amazon.com/images/M/MV5BMTYwOTEwNjAzMl5BMl5BanBnXkFtZTcwODc5MTUwMw@@._V1_SX300.jpg',
 'Production': '20th Century Fox',
 'Rated': 'PG-13',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.8/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '84%'},
  {'Source': 'Metacritic', 'Value': '83/100'}],
 'Released': '18 Dec 2009',
 'Response': 'True',
 'Runtime': '162 min',
 'Title': 'Avatar',
 'Type': 'movie',
 'Website': 'http://www.avatarmov

In [22]:
for key in movie_dict.keys():
    temp_name = movie_dict[key]['Name']
    movie_dict[key]['imdbRating'] = imdb_movies[temp_name]['imdbRating']
    movie_dict[key]['Production'] = imdb_movies[temp_name]['Production']
    movie_dict[key]['Genre'] = imdb_movies[temp_name]['Genre']
    movie_dict[key]['Director'] = imdb_movies[temp_name]['Director']
    #movie_dict[key]['BoxOffice'] = imdb_movies[temp_name]['BoxOffice']
    movie_dict[key]['Awards'] = imdb_movies[temp_name]['Awards']
    movie_dict[key]['Writer'] = imdb_movies[temp_name]['Writer']
    movie_dict[key]['Actors'] = imdb_movies[temp_name]['Actors']
    

In [24]:
#Write CSV file
with open("imdb_movie.csv", "w",newline='') as toWrite:
    writer = csv.writer(toWrite, delimiter=",")
    writer.writerow([ "Rank","Name", "Box Office", "Year","Actor_1","Actor_2","Actor_3","imdbRating","Studio","Genre","Director","Awards","Writer"])
    rank=movie_dict.keys()
    rank= map(int, rank)
    #rank.sort()
    for x in rank:
        x=str(x)
        writer.writerow([x, movie_dict[x]['Name'],
                                 movie_dict[x]['Box Office'], 
                                 movie_dict[x]["Year"],
                                 movie_dict[x]["Actors"].split(',')[0].strip(),
                                 movie_dict[x]["Actors"].split(',')[1].strip(),
                                 movie_dict[x]["Actors"].split(',')[2].strip(),
                                 movie_dict[x]["imdbRating"],
                                 movie_dict[x]["Production"],
                                 movie_dict[x]["Genre"],
                                 movie_dict[x]["Director"],
                                 movie_dict[x]["Awards"],
                                 movie_dict[x]["Writer"]])                                                                  

In [25]:
#Import CSV file to open
import pandas
movie_df = pandas.read_csv('imdb_movie.csv', encoding = 'latin1')
movie_df

,Rank,Name,Box Office,Studio,Year,Actor_1,Actor_2,Actor_3,imdbRating,Production,Genre,Director,Awards,Writer
0,1,Star Wars: The Force Awakens,"$936,662,225",BV,2015,Harrison Ford,Mark Hamill,Carrie Fisher,8.1,Walt Disney Pictures,"Action, Adventure, Fantasy",J.J. Abrams,Nominated for 5 Oscars. Another 57 wins & 123 ...,"Lawrence Kasdan, J.J. Abrams, Michael Arndt, G..."
1,2,Avatar,"$760,507,625",Fox,2009,Sam Worthington,Zoe Saldana,Sigourney Weaver,7.8,20th Century Fox,"Action, Adventure, Fantasy",James Cameron,Won 3 Oscars. Another 85 wins & 128 nominations.,James Cameron
2,3,Titanic,"$658,672,302",Par.,1997,Leonardo DiCaprio,Kate Winslet,Billy Zane,7.8,Paramount Pictures,"Drama, Romance",James Cameron,Won 11 Oscars. Another 110 wins & 75 nominations.,James Cameron
3,4,Jurassic World,"$652,270,625",Uni.,2015,Chris Pratt,Bryce Dallas Howard,Irrfan Khan,7.0,Universal Pictures,"Action, Adventure, Sci-Fi",Colin Trevorrow,7 wins & 57 nominations.,"Rick Jaffa (screenplay by), Amanda Silver (scr..."
4,5,The Avengers,"$623,357,910",BV,2012,Robert Downey Jr.,Chris Evans,Mark Ruffalo,8.1,Walt Disney Pictures,"Action, Adventure, Sci-Fi",Joss Whedon,Nominated for 1 Oscar. Another 38 wins & 79 no...,"Joss Whedon (screenplay), Zak Penn (story), Jo..."
5,6,The Dark Knight,"$534,858,444",WB,2008,Christian Bale,Heath Ledger,Aaron Eckhart,9.0,Warner Bros. Pictures/Legendary,"Action, Crime, Drama",Christopher Nolan,Won 2 Oscars. Another 151 wins & 154 nominations.,"Jonathan Nolan (screenplay), Christopher Nolan..."
6,7,Rogue One: A Star Wars Story,"$532,177,324",BV,2016,Anthony Carboni,Andi Gutierrez,David W. Collins,7.4,NaN,"Reality-TV, Sci-Fi",NaN,NaN,NaN
7,8,Beauty and the Beast,"$504,014,165",BV,2017,Emma Watson,Dan Stevens,Luke Evans,7.3,Walt Disney Pictures,"Family, Fantasy, Musical",Bill Condon,5 wins & 14 nominations.,"Stephen Chbosky (screenplay by), Evan Spilioto..."
8,9,Finding Dory,"$486,295,561",BV,2016,Ellen DeGeneres,Albert Brooks,Ed O'Neill,7.4,Walt Disney Pictures/PIXAR,"Animation, Adventure, Comedy","Andrew Stanton, Angus MacLane(co-director)",Nominated for 1 BAFTA Film Award. Another 15 w...,"Andrew Stanton (original story by), Andrew Sta..."
9,10,Star Wars: Episode I - The Phantom Menace,"$474,544,677",Fox,1999,Liam Neeson,Ewan McGregor,Natalie Portman,6.5,20th Century Fox,"Action, Adventure, Fantasy",George Lucas,Nominated for 3 Oscars. Another 26 wins & 62 n...,George Lucas
